# PyCity Schools Analysis

-  Regarding Reading and Math scores, Reading Scores were generally higher than Math Scores. This observation can be seen within the school (between grades) and on a school-wide basis. 

-  Of the 15 schools analysed, 7 have an unacceptable passing rate (less than 70%) in Math. In these schools, more than 30% of the students failed to score a passing score in Mathematics. 

-  Although every school has an acceptable overall passing rate (greater than 70%), the difference between the best and worst performing school is almost 20 percentage points. 

In [547]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = school_data.merge(student_data, left_on='name',right_on='school',
                                         suffixes=('_School','_Student'))

# Rename columns, drop duplicate school column
# df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
school_data_complete = school_data_complete.rename(columns={
    'name_School': 'School Name',
    'type': 'School Type',
    'size': 'School Size',
    'budget': 'School Budget',
    'name_Student': 'Student Name',
    'gender': 'Gender',
    'grade': 'Grade',
    'reading_score': 'Reading Score',
    'math_score': 'Math Score'
})
school_data_complete = school_data_complete.drop(labels='school',axis=1)
school_data_complete.head()
# print(school_data.columns)
# print(student_data.columns)

,School ID,School Name,School Type,School Size,School Budget,Student ID,Student Name,Gender,Grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [548]:
# Calculate the total number of schools
# count the number of uniques in School Name column
num_schools = school_data_complete['School Name'].nunique()

# Calculate the total number of students
# count the number of uniques in Student ID
num_students = school_data_complete['Student ID'].nunique()

# Calculate the total budget
# count the number of uniques in School budget then sum?
total_budget = school_data_complete['School Budget'].unique().sum()

# Calculate the average math/reading score
avg_math = school_data_complete['Math Score'].mean()
avg_reading = school_data_complete['Reading Score'].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_pass = (avg_math+avg_reading)/2

# Calculate the percentage of students with a passing math score (greater than 70)
# find number of students with 70 or above, then divide by num_students * 100
pass_math = school_data_complete[school_data_complete['Math Score'] > 70]
pass_math = pass_math['Student ID'].nunique() / num_students * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
pass_read = school_data_complete[school_data_complete['Reading Score'] > 70]
pass_read = pass_read['Student ID'].nunique() / num_students * 100

# Create a dataframe to hold the above results
# create dictionary of data and then into dataframe
summary_results = {
    'Total Schools': [num_schools],
    'Total Students': [num_students],
    'Total Budget': [total_budget],
    'Average Math Score': [avg_math],
    'Average Reading Score': [avg_reading],
    '% Passing Math': [pass_math],
    '% Passing Reading': [pass_read],
    '% Overall Passing Rate': [overall_pass]
}
summary_df = pd.DataFrame.from_dict(data=summary_results)
summary_df = summary_df[['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score',
                        '% Passing Math','% Passing Reading','% Overall Passing Rate']]
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,82.971662,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [549]:
# school name as index
group_school = school_data_complete.groupby(['School Name'])

# school type (return value for each school)
s_type = group_school['School Type'].unique()
# total students (count how many students via ID or Name)
tot_students = group_school['Student ID'].nunique()
# total school budget (return for each school)
tot_budget = group_school['School Budget'].unique()
# per student budget (divide total school budget by total students)
per_sb = tot_budget / tot_students
# average math score (mean of math score)
avg_math_school = group_school['Math Score'].mean()
# average reading score (mean of reading score)
avg_reading_school = group_school['Reading Score'].mean()
# % Passing Math (number of students w/ math scores above 70 divided by total students at each school)

# dataframe of all students with Student Name, School Name, Math Score, Reading Score as columns
math_reading_df = pd.DataFrame(data=school_data_complete,columns=['Student Name','School Name',
                                                                   'Math Score','Reading Score'])
# dataframe of only math passing students
math_students_filtered = math_reading_df[math_reading_df['Math Score'] > 70] \
    .drop(columns=['Student Name','Reading Score'])
# grouping of passing students by school
passing_math = math_students_filtered.groupby(['School Name'])
# student count of passing math by school
passing_math = passing_math.count()

tot_students = pd.DataFrame(data=tot_students)
result_df = pd.DataFrame()
result_df['% Passing Math'] = passing_math['Math Score'] / tot_students['Student ID'] * 100

# % Passing Reading (reading scores above 70 divided by total students)
reading_students_filtered = math_reading_df[math_reading_df['Reading Score'] > 70] \
    .drop(columns=['Student Name','Math Score'])
passing_reading = reading_students_filtered.groupby(['School Name'])
passing_reading = passing_reading.count()
result_df['% Passing Reading'] = passing_reading['Reading Score'] / tot_students['Student ID'] * 100
result_df

# Overall Passing Rate (average of last two items)

result_df['% Overall Passing Rate'] = (result_df['% Passing Math'] + result_df['% Passing Reading']) / 2
result_df

# build back dataframe of all information

summary_school_df = pd.DataFrame(data=s_type)

# summary_school_df = pd.DataFrame(data=summary_results_school)
summary_school_df = summary_school_df.join([tot_students,tot_budget,per_sb,avg_math_school,avg_reading_school,
                        result_df['% Passing Math'],result_df['% Passing Reading'], 
                        result_df['% Overall Passing Rate']])

summary_school_df = summary_school_df.rename(columns={'Student ID': 'Students',
                                                     0: 'Budget per Student',
                                                     'Math Score': 'Average Math Score',
                                                     'Reading Score': 'Average Reading Score'})

# fix school type column (comes as [[]]) 

fix_st = np.hstack(summary_school_df['School Type'])
summary_school_df['School Type'] = fix_st

# fix School Budget type column (comes as [[]])
fix_sb = np.hstack(summary_school_df['School Budget'])
summary_school_df['School Budget'] = fix_sb

# fix budget per student column (comes as [[]])
fix_bps = np.hstack(summary_school_df['Budget per Student'])
summary_school_df['Budget per Student'] = fix_bps
summary_school_df

,School Type,Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,71.467098
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,91.686183
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,71.067003


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [550]:
summary_school_df.sort_values(by=['% Overall Passing Rate'],axis=0,ascending=False).head(5)

,School Type,Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,91.686183


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [551]:
summary_school_df.sort_values(by=['% Overall Passing Rate'],axis=0,ascending=True).head(5)

,School Type,Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,71.467098


## Reading and Math Scores by Grade

* Create a table that lists the average Reading and Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

### Note: To increase readibility, all information is collected in one table indexed by School Name and Grade

In [552]:
# groupby school and grade?

group_school_grade_df = school_data_complete[['School Name','Grade','Reading Score','Math Score']]
group_school_grade = group_school_grade_df.groupby(by=['School Name','Grade'])['Reading Score','Math Score'].mean()

# put grouped info into new dataframe
group_school_results = pd.DataFrame(group_school_grade)
group_school_results = group_school_results.rename(columns={'Reading Score': 'Average Reading Score',
                                                     'Math Score': 'Average Math Score',
                                                     })

group_school_results

Average Reading Score  Average Math Score
School Name           Grade                                           
Bailey High School    10th               80.907183           76.996772
                      11th               80.945643           77.515588
                      12th               80.912451           76.492218
                      9th                81.303155           77.083676
Cabrera High School   10th               84.253219           83.154506
                      11th               83.788382           82.765560
                      12th               84.287958           83.277487
                      9th                83.676136           83.094697
Figueroa High School  10th               81.408912           76.539974
                      11th               80.640339           76.884344
                      12th               81.384863           77.151369
                      9th                81.198598           76.403037
Ford High School      10th               81.262712           77.672316
                      11th               80.403642           76.918058
                      12th               80.662338           76.179963
                      9th                80.632653           77.361345
Griffin High School   10th               83.706897           84.229064
                      11th               84.288089           83.842105
                      12th               84.013699           83.356164
                      9th                83.369193           82.044010
Hernandez High School 10th               80.660147           77.337408
                      11th               81.396140           77.136029
                      12th               80.857143           77.186567
                      9th                80.866860           77.438495
Holden High School    10th               83.324561           83.429825
                      11th               83.815534           85.000000
                      12th               84.698795           82.855422
                      9th                83.677165           83.787402
Huang High School     10th               81.512386           75.908735
                      11th               81.417476           76.446602
                      12th               80.305983           77.225641
                      9th                81.290284           77.027251
Johnson High School   10th               80.773431           76.691117
                      11th               80.616027           77.491653
                      12th               81.227564           76.863248
                      9th                81.260714           77.187857
Pena High School      10th               83.612000           83.372000
                      11th               84.335938           84.328125
                      12th               84.591160           84.121547
                      9th                83.807273           83.625455
Rodriguez High School 10th               80.629808           76.612500
                      11th               80.864811           76.395626
                      12th               80.376426           77.690748
                      9th                80.993127           76.859966
Shelton High School   10th               83.441964           82.917411
                      11th               84.373786           83.383495
                      12th               82.781671           83.778976
                      9th                84.122642           83.420755
Thomas High School    10th               84.254157           83.087886
                      11th               83.585542           83.498795
                      12th               83.831361           83.497041
                      9th                83.728850           83.590022
Wilson High School    10th               84.021452           83.724422
                      11th               83.764608           83.195326
                      12th               84.317673   

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
## Note: The Overall Passing Rate is calculated with the Average Math Score and Average Reading Score, but I believe this is an error. It should be calculated by % Passing Math and % Passing Reading

In [553]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# create new dataframe from summary_school_df w/ only relevant information ('Per Student Budget','Average Math Score',
# 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate')
# add a column that plugs in the group name based on the spending per student

scores_spending_df = summary_school_df[['Budget per Student','Average Math Score',
                                     'Average Reading Score','% Passing Math','% Passing Reading']]

scores_spending_df['Spending Ranges (Per Student)'] = pd.cut(scores_spending_df['Budget per Student'],
                                                             spending_bins,labels=group_names)

scores_spending_grouped = scores_spending_df.groupby(by='Spending Ranges (Per Student)')['Average Math Score',
                                                                          'Average Reading Score',
                                                                         '% Passing Math',
                                                                         '% Passing Reading'].mean()

scores_spending_results = pd.DataFrame(scores_spending_grouped)
# scores_spending_results['% Overall Passing Rate'] = (scores_spending_results['% Passing Math'] + \
#                                                     scores_spending_results['% Passing Reading']) / 2
scores_spending_results['% Overall Passing Rate'] = (scores_spending_results['Average Math Score'] + \
                                                    scores_spending_results['Average Reading Score']) / 2

scores_spending_results

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,90.350436,93.325838,83.694607
$585-615,83.599686,83.885211,90.788049,92.410786,83.742449
$615-645,79.079225,81.891436,73.021426,83.214343,80.485330
$645-675,76.997210,81.027843,63.972368,78.427809,79.012526


## Scores by School Size

* Perform the same operations as above, based on school size.

In [554]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# create new dataframe from summary_school_df w/ only relevant information ('Students','Average Math Score',
# 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate')
# add a column that plugs in the group name based on the spending per student

scores_school_df = summary_school_df[['Students','Average Math Score',
                                     'Average Reading Score','% Passing Math','% Passing Reading']]

scores_school_df['School Size'] = pd.cut(scores_school_df['Students'],size_bins,labels=group_names_size)

scores_school_grouped = scores_school_df.groupby(by='School Size')['Average Math Score','Average Reading Score',
                                                                   '% Passing Math','% Passing Reading'].mean()

scores_school_results = pd.DataFrame(scores_school_grouped)
scores_school_results['% Overall Passing Rate'] = (scores_school_results['% Passing Math'] + \
                                                    scores_school_results['% Passing Reading']) / 2
scores_school_results

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


## Scores by School Type

In [555]:
# create new dataframe from summary_school_df w/ only relevant information ('School Type','Average Math Score',
# 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate')
# add a column that plugs in the group name based on the spending per student

scores_schooltype_df = summary_school_df[['School Type','Average Math Score',
                                     'Average Reading Score','% Passing Math','% Passing Reading']]

scores_schooltype_grouped = scores_schooltype_df.groupby(by='School Type')['Average Math Score',
                                                                          'Average Reading Score',
                                                                          '% Passing Math','% Passing Reading'].mean()

scores_schooltype_grouped['% Overall Passing Rate'] = (scores_schooltype_grouped['% Passing Math'] + \
                                                       scores_schooltype_grouped['% Passing Reading']) / 2

scores_schooltype_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
